# PowerShell Basics

Compositing and Bundling Expressions: Starting simple and building bigger things.

* Expressions
* Scripts
* Functions
* Modules

### Expressions

Statements that execute and exist only within a console session.

In [ ]:
$x = 3
$y = 4
$z = $x + $y
$z

### Functions

Named set of expressions which may return a result and/or perform some action. Functions only exist within a console session. Functions may or may not accept input (parameters + data)

The function name is "Add-Numbers" and it has two (2) parameters named X and Y (denoted by $x and $y in the parenthesis)

In [ ]:
function Add-Numbers ( $x, $y ) {
	$x + $y
}

function Add-Numbers2 {
	param ($x, $y)
	$x + $y
}

Test the results...

In [ ]:
Add-Numbers 40 16
Add-Numbers2 40 16

#### Using the Parameter() feature

NOTE: Demonstrate various [parameter()] options and input validation features using the block below.

In [ ]:
function Add-Numbers {
	param (
		[parameter(Mandatory=$True)][int][ValidateRange(1,1000)]$x,
		[parameter()]$y = 0
	)
	$x + $y
}

In [ ]:
Add-Numbers 3
Add-Numbers 3 2
Add-Numbers 1000 5
Add-Numbers 1001 5

### Scripts

Expressions saved into a .PS1 file which can be reused with minimal input changes.

* Expressions within a script will execute upon loading the script.
* Functions and Variables within a script will exist in memory during the script execution only (see "Dot-sourcing" below).
* Global scope Functions and Variables will persist in the console session after script execution completes.

Save the following example into C:\TEMP\MyScript.ps1

In [ ]:
$x = 5
$y = 6
$z = $x + $y
$z

Run the Script within a PowerShell terminal or console:

In [ ]:
& c:\temp\myscript.ps1

Save the following into C:\TEMP\AddStrings.ps1

In [ ]:
function Add-Strings {
	param ($String1, $String2, $Delimiter = ' ')
	($String1, $String2) -join $Delimiter
}

Run the script and try to use the "Add-Strings" function afterwards...

In [ ]:
& c:\temp\AddStrings.ps1

In [ ]:
Add-Strings -String1 "The" -String2 "Car"

"Dot-Sourcing" loads the script into memory until the console session is terminated.

In [ ]:
. c:\temp\AddStrings.ps1

In [ ]:
Add-Strings -String1 "The" -String2 "Car"

### Modules

A Module is a collection of related functions and expressions saved as a .PSM1 file

* Behaves much like dot-sourcing a script with embedded functions
* Allow for using a "manifest" .PSD1 file to describe the module (and more)
* Minimum items: .PSM1 (.PSD1 is optional)

Example Manifest (.psd1)

```powershell
@{
	RootModule        = '.\RetirementTools.psm1'
	ModuleVersion     = '1.0.0'
	GUID              = '5a0e6368-abcd-414d-3210-abcd56787738'
	Author            = 'John Wick'
	CompanyName       = 'The Continental'
	Copyright         = '(c) 2023 Assassasin Inc. All rights reserved.'
	Description       = 'Tools for taking revenge for injuring faithful pets.'
	PowerShellVersion = '5.1'
	...
}
```

#### Module Commands

Command | Description | Notes
--|--|--
Get-Module | List local modules | Available or installed
Find-Module | Search for modules in a repo | PowerShell Gallery
Install-Module | Download a module from a repo | PowerShell Gallery
Update-Module | Update a module to match a repo | Update to latest version or specified version
Import-Module | Import a local module into a console session | Auto-import since PS 5.1
Remove-Module | Remove a module from the console session | Not typically necessary
Uninstall-Module | Delete a module from the local machine or user profile | Removes the local source files

## EXERCISES

### Exercise 1 - List Modules

In [ ]:
# List all modules in current session
Get-Module | Select Name, Version

In [ ]:
# List all modules currently installed
Get-Module -ListAvailable | Select Name, Version

In [ ]:
# Get details about a specific module
Get-Module PSReadLine | Select Name, Version

In [ ]:
# Get details about a specific module from installation path
Get-Module PSReadLine -ListAvailable | Select Name, Version

### Exercise 2 - Get Module Details

In [ ]:
Get-Module PSReadLine -ListAvailable | Select *

In [ ]:
Get-Module PSReadLine -ListAvailable | Select -ExpandProperty Path

* Open the Module path in Windows File Explorer.
* Navigate within the module path and note the folder structure, files and file types.

### Exercise 3 - Search for a Module

In [ ]:
Find-Module Az.Accounts

In [ ]:
# Review the output from this example...
Find-Module Az.Accounts | Select *

In [ ]:
# Search for modules using a wildcard pattern...
Find-Module Az.Auto* | Select -First 5

In [ ]:
# Search for modules using the Tag collection...
Find-Module -Tag azureautomation

In [ ]:
Find-Module -Tag Skatterbrainz

### Exercise 4 - Install a Module

In [ ]:
Install-Module Helium

In [ ]:
# Note that Helium may not appear in the current session list...
Get-Module

In [ ]:
# List the commands (functions) provided in the Helium module...
Get-Command -Module Helium

In [ ]:
# Note that the module now appears because it was implicitly loaded into the current session...
Get-Module

### Exercise 5 - Install a Module with Context

In [ ]:
if (-not (Get-Module ImportExcel -ListAvailable)) {
	Install-Module ImportExcel -Scope CurrentUser
} else {
	Write-Host "ImportExcel is already installed"
}

In [ ]:
Get-Module ImportExcel -ListAvailable | Select-Object Path

### Exercise 6 - Module Dependencies

Compare the output of the following examples.

In [ ]:
Find-Module Az.Automation | Select-Object -ExpandProperty Dependencies

In [ ]:
Find-Module Az.Automation -RequiredVersion 1.6.0 | Select-Object -ExpandProperty Dependencies

### Exercise 7 - Create a Module


1. Create a file named ```MyDates.psm1```, in a convenient location/path. For this exercise we'll use C:\TEMP, but any path is fine.
2. Add 2 or more functions within MyDates.psm1. For example:
   
   ```powershell
   function Get-FutureDate {
	  param (
		[parameter(Mandatory)][datetime]$Date,
		[parameter()][int]$DaysInTheFuture = 0
	  )
      (Get-Date $Date).AddDays($DaysInTheFuture)
   }

   function Get-PastDate {
	  param (
		[parameter(Mandatory)][datetime]$Date,
		[parameter()][int]$DaysBack = 0
	  )
	  (Get-Date $Date).AddDays(-$DaysBack)
   }
   ```
3. Save and close the module file

In [ ]:
if (Test-Path "C:\TEMP\MyDates.psm1") {
	Import-Module C:\TEMP\MyDates.psm1
	Get-Module
} else {
	Write-Warning "Make sure you saved the MyDates.psm1 in the correct path."
}

### Exercise 8 - Create a Module Manifest


Using the same MyDates.psm1 module, we'll create a .psd1 manifest to provide additional details about your module.

In [ ]:
cd C:\TEMP

In [ ]:
New-ModuleManifest -Path C:\TEMP\MyDates.psd1 -RootModule .\MyDates.psm1


* In Windows File Explorer, view the contents of the module directory (e.g. C:\TEMP)
* Open the new MyDates.psd1 file in your editor (Notepad, Visual Studio Code, etc.)
   
   * Notice how every line has a comment above it to explain its purpose.
   * Most properties are commented out. If you didn't specify them in step 3, they use default values.
  
* Close the file in your editor, and Delete the file MyDates.psd1.
* Repeat step 3 but include inputs for the parameters:

   Property | Value
   --|--
   Path | C:\TEMP\MyDates.psd1
   Author | (your name)
   CompanyName | Quisitive
   Copyright | 2023 Quisitive. All rights reserved
   RootModule | .\MyDates.psm1
   ModuleVersion | 1.0.0
   Description | Assorted Date Calculation Tools
   PowerShellVersion | 5.1
   Tags | Dates

In [ ]:
Import-Module C:\TEMP\MyDates.psd1 -Force

In [ ]:
Get-Module MyDates | Select *

## Sharing Modules

Once you have created a Module, you can share it simply by placing the files in a location where others can access it.

If you only wish to share the Module with users who are connected to your organization's network, you can save it to a network file share. Other users can import the Module directlry from the file share, or download it to a local directory. Plan ahead for what will make the most sense when it comes to maintaining the Module later on (e.g. fixing bugs, adding new functions or features)/

If you wish to share your Module with the public, there are some additional considerations:

* Already a Module with the same Name
* Already a Module which covers the same features
* Do the other modules support open source contribution (e.g. GitHub)
* Do you want to allow public contribution

If you cleared all of these considerations, and still wish to make your Module available to the public, you'll need to publish it somewhere which allows for public access, such as PowerShell Gallery (aka PSGallery).

* Create a PowerShell Gallery account
* Obtain an API key from within your PS Gallery account
* Save your API key in a secure, private location (Keeper, LastPass, KeyPass, BitWarden, etc.)
* Publish your module to PowerShell Gallery